# Chapter 19 – Training and Deploying TensorFlow Models at Scale

This notebook contains all the sample code and solutions to the exercises in chapter 19.

## Setup
This project requires Python 3.7 or above:

In [1]:
import sys

assert sys.version_info >= (3, 7)

**Warning**: the latest TensorFlow versions are based on Keras 3. For chapters 10-15, it wasn't too hard to update the code to support Keras 3, but unfortunately it's much harder for this chapter, so I've had to revert to Keras 2. To do that, I set the ```TF_USE_LEGACY_KERAS ```environment variable to ```"1" ```and import the ```tf_keras``` package. This ensures that ```tf.keras``` points to ```tf_keras```, which is Keras 2.*.

In [2]:
IS_COLAB = "google.colab" in sys.modules
if IS_COLAB:
    import os
    os.environ["TF_USE_LEGACY_KERAS"] = "1"
    import tf_keras

And TensorFlow ≥ 2.8:

In [3]:
from packaging import version
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

If running on Colab or Kaggle, you need to install the Google AI Platform client library, which will be used later in this notebook. You can ignore the warnings about version incompatibilities.

* **Warning**: On Colab, you must restart the Runtime after the installation, and continue with the next cells.

In [4]:
import sys
if "google.colab" in sys.modules or "kaggle_secrets" in sys.modules:
    %pip install -q -U google-cloud-aiplatform

This chapter discusses how to run or train a model on one or more GPUs, so let's make sure there's at least one, or else issue a warning:

In [5]:
if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. Neural nets can be very slow without a GPU.")
    if "google.colab" in sys.modules:
        print("Go to Runtime > Change runtime and select a GPU hardware "
              "accelerator.")
    if "kaggle_secrets" in sys.modules:
        print("Go to Settings > Accelerator and select GPU.")

# Serving a TensorFlow Model
Let's start by deploying a model using TF Serving, then we'll deploy to Google Vertex AI.

## Using TensorFlow Serving
The first thing we need to do is to build and train a model, and export it to the SavedModel format.

## Exporting SavedModels
Let's load the MNIST dataset, scale it, and split it.

In [6]:
from pathlib import Path
import tensorflow as tf

# extra code – load and split the MNIST dataset
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# extra code – build & train an MNIST model (also handles image preprocessing)
tf.random.set_seed(42)
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8),
    tf.keras.layers.Rescaling(scale=1 / 255),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

model_name = "my_mnist_model"
model_version = "0001"
model_path = Path(model_name) / model_version
model.save(model_path, save_format="tf")

Epoch 1/10
1719/1719 [==============================] - 3s 1ms/step - loss: 0.6887 - accuracy: 0.8233 - val_loss: 0.3712 - val_accuracy: 0.8996
Epoch 2/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3537 - accuracy: 0.9015 - val_loss: 0.3014 - val_accuracy: 0.9160
Epoch 3/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3052 - accuracy: 0.9141 - val_loss: 0.2681 - val_accuracy: 0.9246
Epoch 4/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.2759 - accuracy: 0.9223 - val_loss: 0.2467 - val_accuracy: 0.9286
Epoch 5/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.2533 - accuracy: 0.9286 - val_loss: 0.2286 - val_accuracy: 0.9344
Epoch 6/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.2345 - accuracy: 0.9340 - val_loss: 0.2144 - val_accuracy: 0.9398
Epoch 7/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.2185 - accuracy: 0.9388 - val_loss: 0.1999 - val_accuracy: